In [39]:
import pandas as pd
import numpy as np
import glob
import os
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [2]:
# load excel 2001-2010 (10 years)
files = glob.glob("S:/Tmp/Data Science/CDC Survey/Data/CDC_CodeCombined/codecombined10years/*.xlsx")
org10years_dict = {}
for f in files:
    org10years_dict[os.path.splitext(os.path.basename(f))[0].split('_')[-1]] = pd.read_excel(f)


In [ ]:
# to improve I/O, dump dict to json - test
# improving training efficiency, dump model

In [3]:
year_list = list(org10years_dict.keys())
for x in year_list:
    print(x, org10years_dict[x].shape)

2005 (375373, 23)
2006 (376328, 23)
2007 (365648, 23)
2008 (165630, 24)
2009 (162151, 24)
2010 (151551, 24)
2001 (330210, 23)
2002 (327255, 23)
2003 (319530, 23)
2004 (370785, 23)


In [28]:
# selected columns
select_col = ['newborn_status', 'sex', 'race', 'unit_age', 'age', 'type_admission', 'diagnosis_combine', 'days_of_care']
cate_col = ['newborn_status', 'sex', 'race', 'type_admission']
int_col = ['age']
code_disease = {'25000': 'Diabetes mellitus', '2724': 'Hyperlipidemia', '3051': 'Tobacco use disorder',
                '4019': 'Hypertension', '41401': 'Coronary atherosclerosis', '42731': 'Atrial fibrillation',
                '4280': 'Congestive heart failure', '53081': 'Esophageal reflux', '5990':'Urinary tract infection',
                'V270': 'Deliver single liveborn', 'V3000': 'Single liveborn delivered without cesarean section'}
diseasepool = list(code_disease.keys()) # 11 selected

In [29]:
def select_trainingset(df_dict, disease_list, selcol_list, diagcol_name):
    """
    input: disease_list, contains one or multiple diagnosis codes, input from dashboard
    output: concated df for preprocessing
    """
    key_list = list(df_dict.keys())
    result_df = pd.DataFrame()
    
    for x in key_list:        
        selcol_df = df_dict[x][selcol_list] # select training columns
        diseasecol_list = selcol_df[diagcol_name].str.split('|').tolist()
        bool_list = [all(elem in e for elem in disease_list) for e in diseasecol_list]
        sub_df = selcol_df[bool_list].reset_index(drop=True)
        result_df = pd.concat([result_df, sub_df], axis=0).reset_index(drop=True)
    
    
    
    return result_df

In [30]:
# udf age preprocess
def age_preprocess(df_select):
    df_age = df_select.copy()
    df_age[['age','unit_age']] = df_age[['age','unit_age']].astype(int)
    df_age.loc[df_age['unit_age'] != 1, 'age'] = 0
    df_ageresult = df_age.drop(['unit_age'], axis=1)
    
    return df_ageresult

In [37]:
# udf astype(pre_define), drop,dummy
def training_preprocess(df, diseasepool, dummy_col, number_col, diagcol_name, ycolname):
    """
    ignore disease out of pool
    output: X(features with dummy 0/1 plus int cols) and y
            component: selected diseases matrix 
    """
    df_preprocess = df.reset_index(drop=True) # reset index
    
    diag_df = pd.DataFrame()
    # diagnosis column
    df_preprocess[diagcol_name] = df_preprocess[diagcol_name].str.strip()
    
    boolmap_dict = {'True': 1, 'False': 0}
    for x in diseasepool:
        bool_list = [x in code_list for code_list in df_preprocess[diagcol_name].str.split('|').tolist()]
        subdiagcol_name = 'diagnosis' + x
        
        diag_df[subdiagcol_name] = pd.Series(bool_list).map(boolmap_dict)
    
    # change type, create dummy
    df_preprocess[dummy_col] = df_preprocess[dummy_col].astype(str)
    df_dummy = pd.get_dummies(df_preprocess[dummy_col], prefix=dummy_col)
    ### get complete dummy columns for each level [TODO]
    ### better: get choices level available
    
    # int columns
    df_preprocess[number_col] = df_preprocess[number_col].astype(int)
    
    # concat
    df_feature = pd.concat([df_dummy, df_preprocess[number_col], diag_df], axis =1)
    
    target = df_preprocess[ycolname].reset_index(drop=True)
    
    return df_feature, target

In [ ]:
# udf prepare the testing case
# get input value from dashboard in format of dictionary
# def testing_case(input_dict)

In [53]:
# udf modeling
def modeling(feature, target):
    """
    output: prediction(take an average), error estimation
    """
    X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=11217)
    
    xgb_param = {'n_estimators': 200, 'eta': 0.1, 'max_depth': 3, 'verbose': -1, 'silent': True}
    regxgb = xgb.XGBRegressor(**xgb_param)
    regxgb.fit(X_train, y_train, eval_metric='rmse')
    
    predxgb = regxgb.predict(X_test)
    pred_table = pd.DataFrame({'y_true': y_test, 'y_predicted': predxgb}).reset_index(drop=True)
    result_avg = round(predxgb.mean(),2)
    rmse = round(sqrt(mean_squared_error(y_test, predxgb)),4)

    return pred_table, result_avg, rmse

In [54]:
# udf wrap function 
def wrap_function(df_dict, disease_list, selcol_list, diseasepool, dummy_col, number_col, diagcol_name = 'diagnosis_combine', ycolname = 'days_of_care'):
    """
    input: disease_list, selected combination;
           diseasepool, top 10/11
    
    """
    
    df_seresult = select_trainingset(df_dict, disease_list, selcol_list, diagcol_name)
    df_afterage = age_preprocess(df_seresult)
    df_feature, target = training_preprocess(df_afterage, diseasepool, dummy_col, number_col, diagcol_name, ycolname)
    pred_table, result_avg, rmse = modeling(df_feature, target)
    
    return pred_table, result_avg, rmse

In [58]:
# udf wrap function 
def wrap_functionv2(disease_list):
    """
    input: disease_list, selected combination;
           diseasepool, top 10/11
    
    """
    
    df_seresult = select_trainingset(org10years_dict, disease_list, select_col, 'diagnosis_combine')
    df_afterage = age_preprocess(df_seresult)
    df_feature, target = training_preprocess(df_afterage, diseasepool, cate_col, int_col, 'diagnosis_combine', 'days_of_care')
    pred_table, result_avg, rmse = modeling(df_feature, target)
    
    return pred_table, result_avg, rmse

In [72]:
def wrap_functionv3(b):
    """
    input: disease_list, selected combination;
           diseasepool, top 10/11
    
    """
    disease_list = selected_data
    
    df_seresult = select_trainingset(org10years_dict, disease_list, select_col, 'diagnosis_combine')
    df_afterage = age_preprocess(df_seresult)
    df_feature, target = training_preprocess(df_afterage, diseasepool, cate_col, int_col, 'diagnosis_combine', 'days_of_care')
    pred_table, result_avg, rmse = modeling(df_feature, target)
    print(result_avg, rmse)
    #return pred_table, result_avg, rmse

In [63]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, HTML, clear_output
from ipywidgets import HBox, VBox
from ipywidgets import Layout, Box
from IPython.display import display

In [73]:
def display_titles():
    display(form)
    display(out)
    b1.on_click(wrap_functionv3)

In [74]:
data = {'25000 - Diabetes mellitus': 'Diabetes mellitus', '2724 - Hyperlipidemia': 'Hyperlipidemia', '3051 - Tobacco use disorder': 'Tobacco use disorder',
  '4019 - Hypertension': 'Hypertension', '41401 - Coronary atherosclerosis': 'Coronary atherosclerosis', '42731 - Atrial fibrillation': 'Atrial fibrillation',
  '4280 - Congestive Heart Failure': 'Congestive heart failure', '53081 - Esophageal reflux': 'Esophageal reflux', '5990 - Urinary tract infection':'Urinary tract infection',
  'V270 - Deliver single liveborn': 'Deliver single liveborn', 'V3000 - Single liveborn delivered without cesarean section': 'Single liveborn delivered without cesarean section'}
names = []
checkbox_objects = []
for key in data:
    checkbox_objects.append(widgets.Checkbox(value=False, description=key))
    names.append(key[:5].strip())

arg_dict = {names[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}




b1 = widgets.Button(description ='Run the Model',button_style='success',icon = 'font-awesome')
form_item_layout = Layout(display='flex', flex_flow='column',justify_content='space-between')
btns2 = [b1]
t1 = widgets.Label(value =r'\(\color{maroon} {'+'INPUT'+'}\)')
txt1 = [t1]
t2 = widgets.Label(value =r'\(\color{maroon} {'+'OUTPUT'+'}\)')
txt2 = [t2]
form_items = [
    Box(children=txt1, layout=form_item_layout),
    Box(children=checkbox_objects, layout=form_item_layout),
    Box(children=txt2, layout=form_item_layout),
    Box(children=btns2, layout=form_item_layout)
]
form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))
selected_data = []
def select_data(**kwargs):
    selected_data.clear()

    for key in kwargs:
        if kwargs[key] is True:
            selected_data.append(key)

    print(selected_data)

out = widgets.interactive_output(select_data, arg_dict)
disease_list = out
display_titles()

Box(children=(Box(children=(Label(value='\\(\\color{maroon} {INPUT}\\)'),), layout=Layout(display='flex', flex…

Output()

3.67 3.8809
